In [2]:
%load_ext iminizinc

from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
# import ipywidgets as widgets
from ipywidgets import *
from IPython.display import display
import IPython
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np

from datetime import datetime
# import talib
import pandas_ta as ta
# from talib.abstract import *
from math import *
from collections import OrderedDict
# import vectorbt as vbt    
import json
from decimal import *
getcontext().prec = 6
import seaborn as sns

import mplfinance as mpf

matplotlib.rcParams.update({'font.size': 5, 'lines.linewidth': 0.5, 'figure.dpi': 300})
plt.rcParams['lines.linewidth'] = 0.5 

import cufflinks as cf

print("Cufflinks Version : {}".format(cf.__version__))
cf.set_config_file(theme='pearl',sharing='public',offline=True)

import bqplot

print("Bqplot Version : {}".format(bqplot.__version__))

<IPython.core.display.Javascript object>

MiniZinc to FlatZinc converter, version 2.7.4, build 863757726
Copyright (C) 2014-2023 Monash University, NICTA, Data61


Cufflinks Version : 0.17.3


Bqplot Version : 0.12.39


In [3]:
odf = pd.read_json(r'../freq-user-data/data/binance/BTC_USDT-8h.json').dropna()
odf.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

odf['date'] = pd.to_datetime(odf['timestamp'], unit='ms', utc=False)
# df.index = df['time']
# df.set_index('time', drop=True, inplace=True)
odf['idate'] = odf.date.dt.strftime('%Y%m%d')
odf.set_index(pd.DatetimeIndex(odf["date"]), inplace=True, drop=True)
# df = df[['time', 'symbol', 'source', 'resolution', 'open', 'high', 'low', 'close', 'volume']]
# df.to_csv (r'./data/binance/BTC_USDT-5m.csv', index = None)
# df.set_index('time')
odf = odf.sort_index()
odf

,timestamp,open,high,low,close,volume,date,idate
date,,,,,,,,
2021-01-01 00:00:00,1609459200000,28923.63,29470.00,28690.17,29092.83,18869.366827,2021-01-01 00:00:00,20210101
2021-01-01 08:00:00,1609488000000,29092.84,29600.00,28872.24,29188.67,20078.654834,2021-01-01 08:00:00,20210101
2021-01-01 16:00:00,1609516800000,29188.67,29360.00,28624.57,29331.69,15234.903350,2021-01-01 16:00:00,20210101
2021-01-02 00:00:00,1609545600000,29331.70,29820.50,28946.53,29750.00,17258.671371,2021-01-02 00:00:00,20210102
2021-01-02 08:00:00,1609574400000,29751.47,31800.00,29473.91,31691.29,43255.778007,2021-01-02 08:00:00,20210102
...,...,...,...,...,...,...,...,...
2023-05-11 00:00:00,1683763200000,27598.74,27622.68,27367.76,27450.07,9819.590800,2023-05-11 00:00:00,20230511
2023-05-11 08:00:00,1683792000000,27450.07,27630.14,26952.00,27179.42,19518.716970,2023-05-11 08:00:00,20230511
2023-05-11 16:00:00,1683820800000,27179.43,27219.52,26702.05,26968.62,18297.005880,2023-05-11 16:00:00,20230511


In [ ]:
layout1 = cf.Layout(
    height=300,
    width=200
)
# print(matplotlib.pyplot.gcf())

fig = odf.figure()
# data=odf.figure()['data']


qf=cf.QuantFig(odf,title='Quant Figure',legend='top',name='GS' )
# qf.figure().axhline(20000)
qf.iplot(dimensions =(1500,500))

fig.add_hline(y=51000)
fig



## Last Swing Low -> Swing High 

In [138]:
df = odf['2023-02-01':]

kl_n = len(df)
kl_close = df.close.astype(int).tolist()
kl_idate = df.idate.to_list()
inow = df.shape[0] 
model_params = {'kl_n': kl_n, 'kl_close': kl_close, 'kl_idate': kl_idate, 'inow': inow}
with open('swing_high.json', 'w+') as f:
    f.write(json.dumps(model_params, indent=0))
    # f.write(f'kl_n = {len(df)};\n')
    # f.write(f'kl_signal = [' + ', '.join(df.close.astype(int).astype(str)) + '];\n')
# kl_idate

In [161]:
from minizinc import Instance, Model, Solver
import nest_asyncio
nest_asyncio.apply()

swing_high = Model("./swing_high.mzn")
# Find the MiniZinc solver configuration for Gecode
gecode = Solver.lookup("gecode")
# Create an Instance of the n-Queens model for Gecode
instance = Instance(gecode, swing_high)

for k,v in model_params.items(): instance[k] = v
    # print(k,v)
# instance["n"] = 4
result = instance.solve()
# # Output the array q
print(result)

Solution(objective=232, ipeak_0=217, ipeak_1=70, itrough_0=115, obj=232, _checker='')


In [120]:
%%minizinc -m bind

array[int] of int: kl_close;

int: kl_n;
set of int: KLN = 1..kl_n;
int: inow;
var int: ipeak_0;
var int: ipeak_1;
var int: itrough_0;

array[1..4] of var int: apktr = [ipeak_1, itrough_0, ipeak_0, inow];

constraint forall(k in apktr) (k >= 1 /\ k <= kl_n);
constraint forall(i in 1..3)(apktr[i+1] - apktr[i] > 3 * 7 * 2);
include "increasing.mzn";
constraint increasing([ipeak_1, itrough_0, ipeak_0]);
include "all_different.mzn";
constraint all_different([ipeak_1, itrough_0, ipeak_0]);
constraint forall(i in itrough_0..ipeak_0)(kl_close[i] >= kl_close[itrough_0] /\ kl_close[i] <= kl_close[ipeak_0]);
constraint forall(i in ipeak_1..itrough_0)(kl_close[i] >= kl_close[itrough_0] /\ kl_close[i] <= kl_close[ipeak_1]);
constraint forall(i in ipeak_0..inow)(kl_close[i] <= kl_close[ipeak_0]);
constraint kl_close[ipeak_0] >= kl_close[ipeak_1];

var int: obj;
constraint obj = inow - ipeak_1;


solve minimize obj;

In [111]:
%%minizinc?

Docstring:
::

  %minizinc [-v] [-s] [-m {return,bind}] [-a] [-t TIME_LIMIT]
                [--solver SOLVER] [--data [DATA ...]]
                [model ...]

MiniZinc magic

positional arguments:
  model                 Model to solve

options:
  -v, --verbose         Verbose output
  -s, --statistics      Output statistics
  -m <{return,bind}>, --solution-mode <{return,bind}>
                        Whether to return solution(s) or bind them to
                        variables
  -a, --all-solutions   Return all solutions for satisfaction problems,
                        intermediate solutions for optimisation problems.
                        Implies -o.
  -t TIME_LIMIT, --time-limit TIME_LIMIT
                        Time limit in milliseconds (includes compilation and
                        solving)
  --solver SOLVER       Solver to run
  --data <[DATA ...]>   Data files
File:      ~/miniconda3/envs/py311/lib/python3.11/site-packages/iminizinc/mzn.py

In [62]:
import mplfinance as fplt
from minizinc import Instance, Model, Solver
import nest_asyncio
nest_asyncio.apply()

fibs = [0, 0.236, 0.382, 0.5, 0.618, 0.786, 1]

dlen = len(odf)

kl_n, kl_close, kl_idate, inow = None,None,None,None
ipeak_1, itrough_0, ipeak_0 = None, None, None
wdf = None

# sl_start= IntSlider(description="Str", min=0, max=dlen, step=10, value=dlen-200)
# sl_end = IntSlider(description="End", min=0, max=dlen, value=dlen, step=10)
sl_range = IntRangeSlider(
    value=[int(dlen-1000), dlen],
    min=0,
    max=dlen,
    step=100,
    description='Rng',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
sl_now = IntSlider(description="Now", min=0, max=dlen, value=dlen, step=50)

# def change_end(val):
#     sl_start.value = sl_end.value - 700
#     sl_end.observe(change_end, 'value')

def range_changed(v):
    wstart, wend = sl_range.value
    sl_now.max = wend-wstart
    # print('Calc wdf.shape: ', wdf.shape)


sl_range.observe(range_changed, 'value')

ui = widgets.VBox([VBox([sl_range, sl_now])])
# wstart, wend
def printer(wrange, inow):
    global kl_n, kl_close, kl_idate
    global ipeak_1, itrough_0, ipeak_0
    global wdf

    wstart, wend = sl_range.value
    wdf = odf.iloc[wstart:wend, :]
    print('inow=',inow)
    if wdf is None: return
    # wdf = odf.iloc[wstart:wend, :]

    kl_n = wdf.shape[0]
    kl_close = wdf.close.astype(int).tolist()
    kl_idate = wdf.idate.to_list()
    
    model_params = {'kl_n': kl_n, 'inow': inow, 'kl_close': kl_close}
    with open('swing_high.json', 'w+') as f: f.write(json.dumps(model_params, indent=0))

    # print('Model params:', model_params)
    swing_high = Model("./swing_high.mzn")
    gecode = Solver.lookup("gecode")
    instance = Instance(gecode, swing_high)

    # instance.branch()
    for k,v in model_params.items(): instance[k] = v
    result = instance.solve()

    print(result)
    i_1, i_2, i_3, obj = resarr = [result['i_1'],result['i_2'],result['i_3'],result['obj']]

    swing_low, swing_high = kl_close[i_2-1], kl_close[i_3-1]
    fib_levels = [swing_low + f*(swing_high-swing_low) for f in fibs]
    # %minizinc swing_high kl_n=kl_n
    
    s = mpf.make_mpf_style(base_mpf_style='charles', rc={'font.size': 8})
    fig = mpf.figure(figsize=(15, 4), style=s) # pass in the self defined style to the whole canvas
    ax = fig.add_subplot(1,1,1) # main candle stick chart subplot, you can also pass in the self defined style here only for this subplot
    # av = fig.add_subplot(2,1,2, sharex=ax)  # volume chart subplot
    # wdf.close.plot(ax=ax, lw=1)

    ################################
    # ax.plot(wdf.close, lw=1)
    # ax.axvline(wdf.index[inow-1], c='b', lw=0.7)
    # ax.axvline(wdf.index[i_1-1], c='g', lw=0.7)
    # ax.axvline(wdf.index[i_3-1], c='g', lw=0.7)
    # ax.axvline(wdf.index[i_2-1], c='r', lw=0.7)
    # ax.hlines(fib_levels, wdf.index[i_2-1], wdf.index[inow-1], lw=0.5)

    ################################
    # for f in fibs:
    #     l = swing_low + f*(swing_high-swing_low)
        

    # for v in [ipeak_0, ipeak_1, itrough_0]: ax.axvline(v-1, lw=0.7)
    ####################
    mpf.plot(wdf, type='candle', ax=ax)
    ax.axvline(inow-1, c='b', lw=0.7)
    # ax.axvline(i_1-1, c='g', lw=0.7)
    ax.axvline(i_3-1, c='g', lw=0.7)
    ax.axvline(i_2-1, c='r', lw=0.7)
    ax.hlines(fib_levels, i_2-1, inow-1, lw=0.5)

out = widgets.interactive_output(printer, {'wrange': sl_range, 'inow': sl_now})
display(ui, out)


Output()

In [55]:
wdf.iloc[100].close

23152.19